## Ejemplos y aplicaciones

Durante esta lección, exploraremos algunos ejemplos de algoritmos variacionales y cómo aplicarlos:

- Cómo escribir un algoritmo variacional personalizado
- Cómo aplicar un algoritmo variacional para encontrar valores propios mínimos
- Cómo utilizar algoritmos variacionales para resolver casos de uso de aplicaciones

### Definiciones de problemas

Imagine que queremos usar un algoritmo variacional para encontrar el valor propio del siguiente observable:

$$
\hat{O}_1 = 2 II - 2 XX + 3 YY - 3 ZZ,
$$

Este observable tiene los siguientes valores propios:

$$
\left\{
\begin{array}{c}
\lambda_0 = -6 \\
\lambda_1 = 4 \\
\lambda_2 = 4 \\
\lambda_3 = 6
\end{array}
\right\}
$$

y estados propios:

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
\end{array}
\right\}
$$

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

## VQE personalizado

Primero exploraremos cómo construir una instancia de VQE manualmente para encontrar el valor propio más bajo para $\hat{O}_1$. Esto incorporará una variedad de técnicas que hemos cubierto a lo largo de este curso. 

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values

Podemos usar esta función de costo para calcular parámetros óptimos

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(8)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

Finalmente, podemos usar nuestros parámetros óptimos para calcular nuestros valores propios mínimos:

In [ ]:
observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

In [ ]:
from numpy.linalg import eigvalsh

solution_eigenvalue = min(eigvalsh(observable.to_matrix()))
print(
    f"Percent error: {abs((experimental_min_eigenvalue - solution_eigenvalue)/solution_eigenvalue):.2e}"
)

Como puede ver, el resultado es extremadamente cercano al ideal. 

## Configuración de VQE de Qiskit

Para mayor comodidad, también podemos usar las implementaciones existentes de Qiskit [VQE](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.html) para encontrar el valor propio más bajo del primer $\hat{O}_1$ observable y observar todos los resultados de salida.

En este caso utilizaremos lo siguiente:

- Comenzaremos a explorar el operador de referencia $\equiv I$ para demostrar los aumentos de velocidad que esto podría proporcionar
- [`Estimator`](https://qiskit.org/documentation/stubs/qiskit.primitives.Estimator.html#qiskit.primitives.Estimator) de Qiskit Terra
- Optimizador [`SLSQP`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.SLSQP.html) (es decir, programación secuencial de mínimos cuadrados)
- Además, estableceremos el punto inicial para el optimizador `SLSQP` en $\vec\theta_0 = (1, \cdots, 1)$.

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
import numpy as np

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)

vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))

Ahora que hemos inicializado la instancia `VQE` , podemos obtener los resultados con el método [`VQE.compute_minimum_eigenvalue`](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue.html#qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue) . Veamos los resultados.

In [ ]:
result = vqe.compute_minimum_eigenvalue(observable)
print(result)

Veamos `optimizer_result` :

In [ ]:
print(result.optimizer_result)

De toda esta información, sin embargo, la parte más importante es el valor propio. Comparémoslo con el valor teórico:

In [ ]:
from numpy.linalg import eigvalsh

eigenvalues = eigvalsh(observable.to_matrix())
min_eigenvalue = eigenvalues[0]

print("EIGENVALUES:")
print(f"  - Theoretical: {min_eigenvalue}.")
print(f"  - VQE: {result.eigenvalue}")
print(
    f"Percent error >> {abs((result.eigenvalue - min_eigenvalue)/min_eigenvalue):.2e}"
)

Como puede ver, el resultado es extremadamente cercano al ideal.

Sin embargo, todavía no hemos analizado los estados propios, ya que no formaban parte de `results` . Para este propósito, vinculemos los valores de los parámetros óptimos `result.optimal_parameters` a `results.optimal_circuit` y definamos un [`Statevector`](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Statevector.html) a partir de ese circuito vinculado (es decir, no parametrizado).

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)
optimal_vector = Statevector(optimal_circuit)

rounded_optimal_vector = np.round(optimal_vector.data, 3)
print(f"EIGENSTATE: {rounded_optimal_vector}")

Este resultado no parece demasiado cercano al teórico de $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) \equiv [\frac{1}{\sqrt{2} },0,0,\frac{1}{\sqrt{2}}]$. Sin embargo, observe que los vectores propios se definen hasta un factor constante. Además, los estados cuánticos siempre están normalizados y son equivalentes hasta una fase global, por lo que podemos verificar fácilmente que estos dos vectores de estado son equivalentes.

In [ ]:
from numpy.linalg import eigh

_, eigenvectors = eigh(observable.to_matrix())
min_eigenvector = eigenvectors.T[0]  # Note: transpose to extract by index

optimal_vector.equiv(min_eigenvector, atol=1e-4)

Podemos concluir que el estado que obtuvimos es equivalente al ideal hasta $10^{-4}$.

### Agregar estado de referencia

En el ejemplo anterior no hemos utilizado ningún operador de referencia $U_R$. Ahora pensemos en cómo se puede obtener el estado propio ideal $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$. Considere el siguiente circuito.

In [ ]:
from qiskit import QuantumCircuit

ideal_qc = QuantumCircuit(2)
ideal_qc.h(0)
ideal_qc.cx(0, 1)

ideal_qc.draw("mpl")

Rápidamente podemos comprobar que este circuito nos da el estado deseado.

In [ ]:
Statevector(ideal_qc)

Ahora que hemos visto cómo se ve un circuito que prepara el estado de la solución, parece razonable usar una puerta de Hadamard como circuito de referencia, de modo que el ansatz completo se convierta en:

In [ ]:
reference = QuantumCircuit(2)
reference.h(0)
# Include barrier to separate reference from variational form
reference.barrier()

ref_ansatz = ansatz.decompose().compose(reference, front=True)

ref_ansatz.draw("mpl")

Para este nuevo circuito, la solución ideal podría alcanzarse con todos los parámetros configurados en $0$, por lo que esto confirma que la elección del circuito de referencia es razonable.

Ahora comparemos el número de evaluaciones de la función de costo, las iteraciones del optimizador y el tiempo empleado con los del intento anterior.

In [ ]:
num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print("NO REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
# You can change the ansatz of the already defined vqe object instead of creating a new one
vqe.ansatz = ref_ansatz

ref_result = vqe.compute_minimum_eigenvalue(observable)

In [ ]:
num_evaluations_ref = ref_result.cost_function_evals
num_iterations_ref = ref_result.optimizer_result.nit
time_ref = ref_result.optimizer_time

print("ADDED REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations_ref}")
print(f"  - Number of iterations: {num_iterations_ref}")
print(f"  - Time: {time_ref:.5f} seconds")
print()

if num_evaluations_ref < num_evaluations:
    print(">> Number of cost function evaluations improved")
elif num_evaluations_ref > num_evaluations:
    print(">> Number of cost function evaluations worsened")
if num_iterations_ref < num_iterations:
    print(">> Number of iterations improved")
elif num_iterations_ref > num_iterations:
    print(">> Number of iterations worsened")
if time_ref < time:
    print(">> Time improved")
elif time_ref > time:
    print(">> Time worsened")

### Cambiar punto inicial

Ahora que hemos visto el efecto de agregar el estado de referencia, veremos qué sucede cuando elegimos diferentes puntos iniciales $\vec{\theta_0}$. En particular usaremos $\vec{\theta_0}=(0,0,0,0,6,0,0,0)$ y $\vec{\theta_0}=(6,6,6,6,6 ,6,6,6,6)$. Recuerde que, como se discutió cuando se introdujo el estado de referencia, la solución ideal se encontraría cuando todos los parámetros son $0$, por lo que el primer punto inicial debería dar menos evaluaciones, iteraciones y tiempo.

In [ ]:
vqe.initial_point = [0, 0, 0, 0, 6, 0, 0, 0]

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
vqe.initial_point = 6 * np.ones(8)

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

## ejemplo VQD

Ahora, en lugar de buscar solo el valor propio más bajo de nuestros observables, buscaremos todos los $4$. Siguiendo la notación del capítulo anterior (así como la de la clase [VQD](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.html) de Qiskit), eso significa $k=4$.

Recuerde que las funciones de costo de VQD son:

$$
C_{l}(\vec{\theta}) := 
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\sum_{j=0}^{l-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2 
\quad \forall l\in\{1,\cdots,k\}=\{1,\cdots,4\}
$$

Esto es particularmente importante porque un vector $\vec{\beta}=(\beta_0,\cdots,\beta_{k-1})$ (en este caso $(\beta_0, \beta_1, \beta_2, \beta_3)$ ) debe pasarse como argumento cuando definimos el objeto `VQD` .

Además, en la implementación de Qiskit de VQD, en lugar de considerar los observables efectivos descritos en el cuaderno anterior, las fidelidades $|\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle |^2$ se calculan directamente a través del algoritmo [`ComputeUncompute`](https://qiskit.org/documentation/stubs/qiskit.algorithms.state_fidelities.ComputeUncompute.html) , que aprovecha una primitiva `Sampler` para muestrear la probabilidad de obtener $|0\rangle$ para el circuito $U_A^\ daga(\vec{\theta})U_A(\vec{\theta^j})$. Esto funciona precisamente porque esta probabilidad es

$$
\begin{aligned}

p_0

& = |\langle 0|U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})|0\rangle|^2 \\[1mm]

& = |\big(\langle 0|U_A^\dagger(\vec{\theta})\big)\big(U_A(\vec{\theta^j})|0\rangle\big)|^2 \\[1mm]

& = |\langle \psi(\vec{\theta}) |\psi(\vec{\theta^j}) \rangle|^2 \\[1mm]

\end{aligned}
$$

Finalmente, para probar un nuevo optimizador, usemos [`COBYLA`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.COBYLA.html) en lugar de `SLSQP` .

In [ ]:
from qiskit.primitives import Sampler
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.algorithms.eigensolvers import VQD

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 4
betas = [40, 60, 30, 30]

vqd = VQD(estimator, fidelity, ansatz, optimizer, k=k, betas=betas)

Como la única diferencia de API con los ejemplos anteriores, observe que en lugar de llamar al método `VQE.compute_minimum_eigenvalue` , llamaremos a [`VQD.compute_eigenvalues`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.compute_eigenvalues.html) ​​. Comencemos examinando el siguiente observable:

$$
\hat{O}_2 := 2 II - 3 XX + 2 YY - 4 ZZ
$$

Este observable tiene los siguientes valores propios:

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

y estados propios:

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)
\end{array}
\right\}
$$

In [ ]:
observable_2 = SparsePauliOp.from_list([("II", 2), ("XX", -3), ("YY", 2), ("ZZ", -4)])

result = vqd.compute_eigenvalues(observable_2)
print(result)

El [`VQDResult`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQDResult.html) que obtenemos es completamente análogo al `VQEResult` . Sólo se diferencia en que cada atributo es una lista cuyo $i$-ésimo elemento corresponde al $i$-ésimo valor propio.

Ahora que hemos visto los valores propios, comparemos los vectores propios experimentales con los teóricos:

In [ ]:
optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Estos resultados son los mismos que los esperados excepto por un pequeño error de aproximación y fase global.

Ahora resolvamos este problema para el primer $\hat{O}_1 := 2 II - 2 XX + 3 YY - 3 ZZ$ observable.

In [ ]:
result = vqd.compute_eigenvalues(observable)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Aquí los estados propios correspondientes a $\lambda_1 = \lambda_2 = 4$ no son los mismos que los esperados:

$$
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle) \\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
$$

Esto sucede precisamente porque como $\lambda_1=\lambda_2$, cualquier combinación lineal compleja también es un estado propio con el mismo valor propio:

$$
\begin{aligned}
\alpha_1 |\phi_1\rangle + \alpha_2 |\phi_2\rangle
& = \frac{1}{\sqrt{2}}(\alpha_1 |00\rangle + \alpha_2 |01\rangle - \alpha_2 |10\rangle - \alpha_1 |11\rangle) \\[1mm]
& \equiv \frac{1}{\sqrt{2}}[\alpha_1, \alpha_2, -\alpha_2, -\alpha_1]
\end{aligned}
$$

Eso es exactamente lo que estamos viendo con estos resultados.

### Cambiar versiones beta

Como se mencionó en la lección de instancias, los valores de $\vec{\beta}$ deberían ser mayores que la diferencia entre los valores propios. Veamos qué pasa cuando no cumplen esa condición con $\hat{O}_2$

$$
\hat{O}_2 = 2 II - 3 XX + 2 YY - 4 ZZ
$$

con valores propios

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

In [ ]:
vqd.betas = np.ones(4)
result = vqd.compute_eigenvalues(observable_2)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Esta vez, el optimizador devuelve el mismo estado $|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ como una solución propuesta para todos los estados propios: que es claramente equivocado Esto sucede porque las betas eran demasiado pequeñas para penalizar el estado propio mínimo en las sucesivas funciones de costo. Por lo tanto, no se excluyó del espacio de búsqueda efectivo en iteraciones posteriores del algoritmo, y siempre se eligió como la mejor solución posible.

Recomendamos experimentar con los valores de $\vec{\beta}$ y asegurarse de que sean mayores que la diferencia entre los valores propios. 

## Química cuántica: Solucionador de estado fundamental y energía excitada

Nuestro objetivo es minimizar el valor esperado de la energía representativa observable (Hamiltoniana $\hat{\mathcal{H}}$):

$$
\min_{\vec\theta} \langle\psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle
$$

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()

mapper = JordanWignerMapper()

h2_reference_state = HartreeFock(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
)

ansatz = UCC(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
    initial_state=h2_reference_state,
    excitations=2,
)

ansatz.decompose().decompose().draw("mpl")

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Sampler, Estimator
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

estimator = Estimator()
sampler = Sampler()
fidelity = ComputeUncompute(sampler)

vqe = VQE(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)
gse = GroundStateEigensolver(qubit_mapper=mapper, solver=vqe)
result = gse.solve(h2_problem)

print(result)

También podemos aprovechar VQD para resolver los estados excitados

In [ ]:
from qiskit.algorithms.eigensolvers import VQD

h2_operators, _ = gse.get_qubit_operators(h2_problem)

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 3
betas = [33, 33, 33]

vqd = VQD(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    fidelity=fidelity,
    initial_point=np.zeros(ansatz.num_parameters),
    k=k,
    betas=betas,
)
result = vqd.compute_eigenvalues(operator=h2_operators)
vqd_values = result.optimal_values
print(vqd_values)

## Optimización: Max-Cut

El problema de corte máximo (Max-Cut) es un problema de optimización combinatoria que consiste en dividir los vértices de un gráfico en dos conjuntos disjuntos de modo que se maximice el número de aristas entre los dos conjuntos. Más formalmente, dado un gráfico no dirigido $G=(V,E)$, donde $V$ es el conjunto de vértices y $E$ es el conjunto de aristas, el problema Max-Cut pide dividir los vértices en dos subconjuntos disjuntos , $S$ y $T$, tal que se maximiza el número de aristas con un extremo en $S$ y el otro en $T$.

Podemos aplicar Max-Cut para resolver varios problemas, incluidos: agrupamiento, diseño de redes, transiciones de fase, etc. Comenzaremos creando un gráfico de problemas:

In [ ]:
import networkx as nx

n = 4
G = nx.Graph()
G.add_nodes_from(range(n))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
G.add_weighted_edges_from(edge_list)

colors = ["red" for i in range(n)]


def draw_graph(G, colors):
    """Draws the graph with the chose colors"""
    layout = nx.shell_layout(G)
    nx.draw_networkx(G, node_color=colors, pos=layout)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)


draw_graph(G, colors)

Para acceder cómodamente a los pesos de todas las aristas, puedes utilizar la matriz de adyacencia. Esta matriz se puede obtener con [networkx.to_numpy_array()](https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.to_numpy_array.html) .

In [ ]:
w = nx.to_numpy_array(G)
print(w)

El módulo [`qiskit_optimization`](https://qiskit.org/documentation/optimization/index.html) incluye una clase llamada [`Maxcut`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.html) que le permite definir un problema Max-Cut ponderado a partir de la matriz de adyacencia $w$. A partir de ese problema podemos obtener el problema de optimización binaria equivalente que deducimos en la sección anterior con [`Maxcut.to_quadratic_program()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.to_quadratic_program.html)

In [ ]:
from qiskit_optimization.applications import Maxcut

max_cut = Maxcut(w)

quadratic_program = max_cut.to_quadratic_program()
print(quadratic_program.prettyprint())

Este problema se puede expresar como un problema de optimización binaria. Para cada nodo $0 \leq i &lt; n$, donde $n$ es el número de nodos del grafo (en este caso $n=4$), consideraremos la variable binaria $x_i$. Esta variable tendrá el valor $1$ si el nodo $i$ es uno de los grupos que etiquetaremos como $1$ y $0$ si está en el otro grupo, que etiquetaremos como $0$. También denotaremos como $w_{ij}$ (elemento $(i,j)$ de la matriz de adyacencia $w$) el peso de la arista que va del nodo $i$ al nodo $j$. Como el gráfico no está dirigido, $w_{ij}=w_{ji}$. Entonces podemos formular nuestro problema como maximizando la siguiente función de costo:

$$
\begin{aligned}
C(\vec{x})
& =\sum_{i,j=0}^n w_{ij} x_i(1-x_j)\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i,j=0}^n w_{ij} x_ix_j\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i=0}^n \sum_{j=0}^i 2w_{ij} x_ix_j
\end{aligned}
$$

Para resolver este problema con una computadora cuántica, vamos a expresar la función de costo como el valor esperado de un observable. Sin embargo, los observables que Qiskit admite de forma nativa consisten en operadores de Pauli, que tienen valores propios $1$ y $-1$ en lugar de $0$ y $1$. Por eso vamos a hacer el siguiente cambio de variable:

Donde $\vec{x}=(x_0,x_1,\cdots,x_{n-1})$. Podemos usar la matriz de adyacencia $w$ para acceder cómodamente a los pesos de todas las aristas. Esto se utilizará para obtener nuestra función de costo:

$$
z_i = 1-2x_i \rightarrow x_i = \frac{1-z_i}{2}
$$

Esto implica que:

$$
\begin{array}{lcl} x_i=0 & \rightarrow & z_i=1 \\ x_i=1 & \rightarrow & z_i=-1.\end{array}
$$

Entonces, la nueva función de costo que queremos maximizar es:

$$
\begin{aligned}
C(\vec{z})
& = \sum_{i,j=0}^n w_{ij} \bigg(\frac{1-z_i}{2}\bigg)\bigg(1-\frac{1-z_j}{2}\bigg)\\[1mm]

& = \sum_{i,j=0}^n \frac{w_{ij}}{4} - \sum_{i,j=0}^n \frac{w_{ij}}{4} z_iz_j\\[1mm]

& = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j
\end{aligned}
$$

Además, la tendencia natural de una computadora cuántica es encontrar mínimos (generalmente la energía más baja) en lugar de máximos, por lo que en lugar de maximizar $C(\vec{z})$ vamos a minimizar:

$$
-C(\vec{z}) =  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Ahora que tenemos una función de costo a minimizar cuyas variables pueden tener los valores $-1$ y $1$, podemos hacer la siguiente analogía con la Pauli $Z$:

$$
z_i \equiv Z_i = \overbrace{I}^{n-1}\otimes ... \otimes \overbrace{Z}^{i} \otimes ... \otimes \overbrace{I}^{0}
$$

En otras palabras, la variable $z_i$ será equivalente a una puerta $Z$ actuando sobre el qubit $i$. Además:

$$
Z_i|x_{n-1}\cdots x_0\rangle = z_i|x_{n-1}\cdots x_0\rangle \rightarrow \langle x_{n-1}\cdots x_0 |Z_i|x_{n-1}\cdots x_0\rangle = z_i
$$

Entonces el observable que vamos a considerar es:

$$
\hat{H} = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} Z_iZ_j
$$

a lo que tendremos que añadir después el término independiente:

$$
\texttt{offset} = - \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Esta transformación se puede hacer con [`QuadraticProgram.to_ising()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.QuadraticProgram.to_ising.html) .

In [ ]:
observable, offset = quadratic_program.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(observable))

Podemos usar una instancia de VQE para encontrar los parámetros óptimos de la siguiente manera:

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
import numpy as np

ansatz = TwoLocal(observable.num_qubits, "rx", reps=1)
optimizer = COBYLA()

vqe = VQE(
    estimator=Estimator(),
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)

result = vqe.compute_minimum_eigenvalue(observable)

print(result)

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)

x = max_cut.sample_most_likely(Statevector(optimal_circuit))
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", quadratic_program.objective.evaluate(x))

# plot results
colors = ["red" if x[i] == 0 else "orange" for i in range(n)]
draw_graph(G, colors)

Con esta lección, has aprendido:

- Cómo escribir un algoritmo variacional personalizado
- Cómo aplicar un algoritmo variacional para encontrar valores propios mínimos
- Cómo utilizar algoritmos variacionales para resolver casos de uso de aplicaciones

¡Continúe con la lección final para realizar su evaluación y ganar su insignia!